In [6]:
from collections import namedtuple
from decimal import Decimal
import inspect

import arrow

from django.db import connection, connections, models, reset_queries


def show_class(class_):
    for f in class_._meta.fields:
        print(f.name)


def show_obj(obj):
    for f in obj._meta.model._meta.fields:
        print('{} --> {}'.format(f.name, getattr(obj, f.name)))
        

def query_count():
    count = sum(len(c.queries) for c in connections.all())
    reset_queries()
    return count

def show_func_code(f):
    print(inspect.getsource(f))

%load_ext autoreload
%autoreload 2

In [7]:
foodInterface = namedtuple('foodInterface', 'cals, fat, carbs, protein')
FoodMacros = namedtuple("FoodMacros", "cals, fat, carbs, sugar, protein")
FoodMacrosPerGram = namedtuple("FoodMacrosPerGram", "cals_per_gram, fat_per_gram, carbs_per_gram, sugar_per_gram, protein_per_gram")

class FoodCombo:
    def __init__(self, food, grams=None):
        self.food = food
        if grams:
            self.grams = grams
        else:
            self.grams = food.grams
            
        self.cals = food.cals
        self.fat = food.fat
        self.carbs = food.carbs
        self.sugar = food.sugar
        self.protein = food.protein
        
    @property
    def macros(self):
        cals = self.cals
        fat = self.fat / 9
        carbs = self.carbs / 4
        sugar = self.sugar / 4
        protein = self.protein / 4
        return FoodMacros(cals=cals, fat=fat, carbs=carbs, sugar=sugar, protein=protein)

    @property
    def macros_per_gram(self):
        cals_per_gram = self.cals / self.grams
        fat_per_gram = self.fat / self.grams
        carbs_per_gram = self.carbs / self.grams
        sugar_per_gram = self.sugar / self.grams
        protein_per_gram = self.protein / self.grams
        return FoodMacrosPerGram(
            cals_per_gram=cals_per_gram,
            fat_per_gram=fat_per_gram,
            carbs_per_gram=carbs_per_gram,
            sugar_per_gram=sugar_per_gram,
            protein_per_gram=protein_per_gram
        )

    def __add__(self, otherFoodCombo):
        thisFood = self.food
        otherFood = otherFoodCombo.food
        total_grams = self.grams + otherFoodCombo.grams
        cals = thisFood.cals + otherFood.cals
        fat = thisFood.fat + otherFood.fat
        carbs = thisFood.carbs + otherFood.carbs
        sugar = thisFood.sugar + otherFood.sugar
        protein = thisFood.protein + otherFood.protein
        print(cals, fat, carbs, sugar, protein)
        new_food = FoodMacros(cals=cals, fat=fat, carbs=carbs, sugar=sugar, protein=protein)
        new_food_combo = FoodCombo(new_food, grams=total_grams)
        return new_food_combo 
    
    def __repr__(self):
        food = self.food
        return f"Cals: {food.cals}, Fat: {food.fat}, Carbs: {food.carbs}, Sugar: {self.sugar}, Protein: {food.protein}"